In [19]:
import numpy as np


In [20]:
from product import Product
from json import load

with open('agora_hack_products.json', encoding='utf-8') as file:
    all_products = load(file)

all_products = [Product(**p) for p in all_products]

In [21]:
references = [p for p in all_products if p.is_reference]
references_id_set = set([ref.product_id for ref in references])
products = [p for p in all_products if p.product_id not in references_id_set]

In [22]:
from sklearn.model_selection import train_test_split

# удалим часть эталонов, что бы в датасете были 'ничейные' товары
references, nulled_references = train_test_split(
    references, test_size=0.5, random_state=42
)

nulled_references_set = set([r.product_id for r in nulled_references])
for p in products:
    if p.reference_id in nulled_references_set:
        p.reference_id = None

products_train, products_test = train_test_split(
    products, test_size=0.5, random_state=42
)
all_products = products_train + references
products_test = products_test + nulled_references

In [23]:
def accuracy(predicted, target):
    return list(map(lambda v: v[0] == v[1], zip(predicted, target))).count(True) / len(predicted)

In [29]:
from sklearn.svm import OneClassSVM, LinearSVC, SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import RidgeClassifier
from spacy.lang.en import STOP_WORDS as EN_STOP_WORDS
from spacy.lang.ru import STOP_WORDS as RU_STOP_WORDS

from BaseSVDD import BaseSVDD

from model import ProductMatchingModel



model = ProductMatchingModel(RidgeClassifier(alpha=0.4), RidgeClassifier(alpha=0.1), RidgeClassifier(alpha=0.1),
                             TfidfVectorizer(stop_words=RU_STOP_WORDS or EN_STOP_WORDS))

model.fit(all_products)

print(accuracy(model.predict(all_products), map(lambda p: p.reference_id, all_products)),
accuracy(model.predict(products_test), map(lambda p: p.reference_id, products_test)))

0.8547692307692307 0.8394833948339483


In [25]:
def classify_error(pred, targ):
    if pred == targ:
        return
    if pred is None and targ is not None:
        return 'freerider'
    if pred is not None and targ is None:
        return 'bastard'
    return 'chameleon'


In [26]:
from collections import Counter


results = list(zip(model.predict(products_test), map(lambda p: p.reference_id, products_test)))
cnt = Counter([classify_error(a, b) for a,b in results if a != b])
cnt

Counter({'bastard': 190, 'freerider': 44, 'chameleon': 16})

In [27]:
0/0

ZeroDivisionError: division by zero

In [ ]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.base import ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import clone

from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en import STOP_WORDS as EN_STOP_WORDS
from spacy.lang.ru import STOP_WORDS as RU_STOP_WORDS

import pandas as pd
import plotly.express as px

from model import ProductMatchingModel

use_models = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

models = list(zip(map(lambda m: m.__class__.__name__, use_models), use_models)) # now its [(name, model), ...]

model_tests_df = []
for name, model in models:
    master_model = ProductMatchingModel(RidgeClassifier(alpha=1.9), clone(model), clone(model),
                                        vectorizer=TfidfVectorizer(stop_words=RU_STOP_WORDS or EN_STOP_WORDS))
    master_model.fit(all_products)
    train_acc, test_acc = accuracy(master_model.predict(all_products), map(lambda p: p.reference_id, all_products)), \
                          accuracy(master_model.predict(products_test), map(lambda p: p.reference_id, products_test)),

    print(name, f"train: {train_acc}, test: {test_acc}")
    model_tests_df.append([name, train_acc, test_acc])

model_tests_df = pd.DataFrame(model_tests_df, columns=['name', 'train_acc', 'test_acc'])

bar = px.bar(model_tests_df, x='name', y=['train_acc', 'test_acc'], barmode='group', log_y=True)
bar.show()